<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_3_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [7]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    658      0 --:--:-- --:--:-- --:--:--   656
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [8]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthritis
4,8,arthritis


In [14]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-3.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,5,rapid heart beat
2,6,diastolic dysfunction
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [19]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.137566,4.227102,0.267634,03:59


In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.049339,3.675897,0.335077,03:55


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.759856,3.635488,0.340497,03:55


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.725279,3.599182,0.344802,03:56


In [0]:
learn.save_encoder('bigtweet_fold3_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold3_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.600710,4.524093,0.292621,01:09


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.828610,4.121619,0.335878,01:20


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.578058,3.901814,0.360051,03:21


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.372766,3.744385,0.384224,03:22
1,3.170944,3.643758,0.389313,03:27


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.067037,3.547116,0.407125,03:23
1,2.990181,3.493335,0.431298,03:26


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.972757,3.480484,0.431298,03:21
1,2.846744,3.415337,0.441476,03:20


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.819133,3.422604,0.436387,03:22
1,2.698880,3.388197,0.440204,03:21


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.691094,3.400133,0.437659,03:17
1,2.605834,3.387208,0.450382,03:20


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.572134,3.340398,0.450382,03:31
1,2.624488,3.349302,0.451654,03:15
2,2.487217,3.357652,0.459288,03:20
3,2.477941,3.372388,0.450382,03:27


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.404047,3.409365,0.455471,03:37
1,2.517848,3.410770,0.422392,03:26
2,2.460502,3.348855,0.456743,03:25
3,2.350333,3.360704,0.424936,03:21


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.410564,3.391140,0.421120,03:16
1,2.447483,3.383711,0.455471,03:09
2,2.353487,3.380784,0.465649,03:21
3,2.272173,3.378505,0.458015,03:29


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.357906,3.430572,0.438931,03:28
1,2.382393,3.315882,0.454198,03:29
2,2.292077,3.358436,0.461832,03:31
3,2.200632,3.391396,0.456743,03:22


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.297776,3.503504,0.437659,03:26
1,2.289914,3.475813,0.427481,03:26
2,2.223364,3.418626,0.422392,03:31
3,2.166360,3.445652,0.423664,03:11


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.195258,3.392268,0.460560,03:23
1,2.252756,3.444355,0.464377,03:22
2,2.256117,3.544893,0.459288,03:14
3,2.129530,3.431436,0.463104,03:23


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.259524,3.573381,0.455471,03:21
1,2.266873,3.497324,0.458015,03:26
2,2.171664,3.499456,0.454198,03:14
3,2.244674,3.634749,0.455471,03:14


In [0]:
# save the best model

learn.save_encoder('ask-3-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [45]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[37, 5, 1035, 1035, 1035, 1035, 49, 1024, 178, 27, 27, 598, 1035, 635, 414, 39, 896, 214, 864, 864, 864, 864, 864, 864, 1035, 415, 775, 49, 49, 82, 214, 0, 473, 214, 291, 214, 214, 214, 214, 214, 355, 355, 355, 164, 61, 61, 810, 214, 896, 418, 70, 1035, 517, 418, 427, 289, 289, 864, 164, 741, 741, 741, 418, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 289, 418, 418, 289, 80, 80, 80, 89, 214, 418, 418, 94, 1026, 94, 94, 318, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 567, 741, 317, 798, 289, 214, 318, 214, 214, 861, 418, 183, 877, 864, 289, 1035, 864, 636, 364, 315, 227, 896, 418, 517, 214, 214, 125, 125, 125, 125, 125, 125, 125, 289, 343, 861, 343, 343, 622, 473, 473, 473, 140, 344, 447, 164, 1035, 475, 630, 3, 418, 418, 167, 1035, 183, 289, 741, 289, 260, 686, 182, 173, 418, 704, 686, 864, 289, 289, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 240, 240, 240, 240, 240, 240, 745, 190, 190, 190, 190

In [46]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

186
0.21478060046189376
